# Fraud Detection Using Multilayer Perceptron
### Define Model

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(29 , 20)
        self.fc2 = nn.Linear(20 , 10)
        self.fc3 = nn.Linear(10 , 2)
    def forward(self,x):
        #Layer 1
        x = self.fc1(x)
        x = F.relu(x)
        
        #Layer 2
        x = self.fc2(x)
        x = F.relu(x)
        
        #Layer 3
        x = self.fc3(x)
        
        return x

### Load Data

In [15]:
import pandas as pd
df = pd.read_csv('../input/fraud-detection/creditcard.csv')
df.drop('Time',axis = 1, inplace = True)

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)
out = []
for t in [train,test]:
  target = np.array(t['Class'].values)
  target_t = torch.tensor(target)
  torch.reshape(target_t, (1, len(target)))
  features = np.array(t.drop('Class', axis=1).values)
  features_t = torch.tensor(features)
  tens = torch.utils.data.TensorDataset(target_t, features_t)
  out.append(tens)


# Passing to DataLoader
train_loader = torch.utils.data.DataLoader(out[0], shuffle=True, batch_size = 100)
test_loader = torch.utils.data.DataLoader(out[1], shuffle=False, batch_size = 100)
print(len(train_loader))
print(len(test_loader))

1994
855


### Define model and loss functions

In [22]:
# Instantiate model  
model = MLP()  

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005) 

### Train Model

In [26]:
from tqdm.notebook import tqdm, trange

for labels, values ,  in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = values  
        y = model(x.float())
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

  0%|          | 0/1994 [00:00<?, ?it/s]

Testing

In [27]:
## Testing
correct = 0
total = len(test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for  labels, values in tqdm(test_loader):
        # Forward pass
        x = values
        y = model(x.float())
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())
    
print('Test accuracy: {}'.format(correct/total))

  0%|          | 0/855 [00:00<?, ?it/s]

Test accuracy: 0.9992509484291077
